## Students Details

- *Name*: Himani Thakkar
- *Student #*: 202292672
- *Email*: hthakkar@mun.ca

---

- *Name*: Pranav Arora
- *Student #*: 202286040
- *Email*: parora@mun.ca

---


In [20]:
# To run the notebook on voila use: $ voila .\Project2.ipynb

# Sometimes it takes long execution time, which is also addressed in its documentation (https://voila.readthedocs.io/en/stable/customize.html#preheated-kernels).
# To migitage the long execution times, they have deviced a solution: run: $ voila --preheat_kernel=True --pool_size=5
# Then you can choose Project2.ipynb on the voila dashboard.

# There's a little hack that we discovered, refresh your browser each time it gets stucked, you'll probably have to do that thrice, but it is much faster.

<div style="text-align:center; margin:auto;">
<h1 style="font-weight: bolder;">Ischemic Stroke Visualization Dashboard</h1>
</div>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import squarify

import ipywidgets as widgets
from ipywidgets import interact, widgets, HBox, VBox
from ipywidgets import interactive_output
from IPython.display import HTML, display


import warnings
warnings.filterwarnings('ignore')

In [3]:
RED = '#CA0016'
BLUE = '#009797'
OCHRE = '#E0D897'

## Patient Demographic Analysis

In [4]:
df = pd.read_csv('stroke.csv')

In [5]:
df_1 = df.copy()

In [6]:
discharge_time = df_1['Discharge Time']
discharge_time = pd.to_datetime(discharge_time)

admission_time = df_1['Admission Time']
admission_time = pd.to_datetime(admission_time)

df_1['Admit Duration'] = discharge_time - admission_time

emergency_time = df_1['Emergency Dept Time']
emergency_time = pd.to_datetime(emergency_time)

ct_time = df_1['CT Scan Time']
ct_time = pd.to_datetime(ct_time)

df_1['Emergency Response Time'] = ct_time - emergency_time

@interact(Gender=widgets.Select(options=['All', 'Female', 'Male'], value='All'), Factor=widgets.Select(options=['Age','Emergency Response Time', 'Admit Duration'], value='Age'), Show_Comorbidities=True)
def plot_demographic(Gender, Factor, Show_Comorbidities):

    if Gender=='Female':
        df_demo = df_1[df_1['Gender'] == 'F']
    elif Gender =='Male':
        df_demo =df_1[df_1['Gender'] == 'M']
    else:
        df_demo = df_1
    
    df_com = df_demo[df_demo['Comorbidities']==True]
    df_com_not = df_demo[df_demo['Comorbidities']==False]

    fig, ax = plt.subplots(figsize=(10,6))

    if Show_Comorbidities:
        if Factor=='Age':
            sns.histplot(df_com[Factor], label='Comorbidities', element='step', stat='frequency',ec='black', color=RED) 
            sns.histplot(df_com_not[Factor], label='Non-Comorbidities', element='step', stat='frequency',ec='black', color=BLUE)
            ax.set_xlabel('Age (years)') 
        else:
            if Factor == 'Admit Duration':
                sns.histplot(df_com[Factor].dt.total_seconds() / 86400, label='Comorbidities', element='step',ec='black', stat='frequency', color=RED)
                sns.histplot(df_com_not[Factor].dt.total_seconds() / 86400, label='Non-Comorbidities', element='step',ec='black', stat='frequency', color=BLUE)
                ax.set_xlabel('Admit Time (days)')
            else:
                sns.histplot(df_com[Factor].dt.total_seconds() / 60, label='Comorbidities', element='step',ec='black', stat='frequency', color=RED)
                sns.histplot(df_com_not[Factor].dt.total_seconds() / 60, label='Non-Comorbidities', element='step',ec='black', stat='frequency', color=BLUE)
                ax.set_xlabel('Response Time (minutes)')
    else:
        if Factor=='Age':
            sns.histplot(df_demo[Factor], label=Factor, element='step', color=OCHRE)
            ax.set_xlabel('Age (years)') 
        else:
            if Factor == 'Admit Duration':
                sns.histplot(df_demo[Factor].dt.total_seconds() / 86400, label=Factor, element='step',ec='black', stat='frequency', color=OCHRE)
                ax.set_xlabel('Admit Time (days)')
            else:
                sns.histplot(df_demo[Factor].dt.total_seconds() / 60, label=Factor, element='step',ec='black', stat='frequency', color=OCHRE)
                ax.set_xlabel('Response Time (minutes)')
        
    ax.legend()
    ax.set_ylabel('Patient Frequency')
    ax.set_title(f'{Gender} Demographic Data ({Factor})')
    plt.show()
    

interactive(children=(Select(description='Gender', options=('All', 'Female', 'Male'), value='All'), Select(des…

## Stay Analysis

In [7]:
df_2 = df.copy()

In [8]:
def categorize_age(age):
    if age <= 45:
        return '0-45'
    elif 45 < age <= 60:
        return '45-60'
    elif 60 < age <=70:
        return '60-70'
    elif 70 < age <= 80:
        return '70-80'
    else:
        return '80+'

df_2['Age Group'] = df_2['Age'].apply(categorize_age)

df_2['ICU Checkout Time'] = pd.to_datetime(df_2['ICU Checkout Time'])
df_2['ICU Arrival Time'] = pd.to_datetime(df_2['ICU Arrival Time'])
df_2['ICU Stay (Days)'] = (df_2['ICU Checkout Time'] - df_2['ICU Arrival Time']).dt.total_seconds() / (60 * 60 * 24)


df_2['Admission Time'] = pd.to_datetime(df_2['Admission Time'])
df_2['Discharge Time'] = pd.to_datetime(df_2['Discharge Time'])
df_2['HospitalStay'] = (df_2['Discharge Time'] - df_2['Admission Time']).dt.total_seconds() / (60 * 60 * 24)

base_color = RED

Gender_widget = widgets.Select(options=['All', 'Female', 'Male'], value='All', description='Gender')
Comorbidities_widget = widgets.Checkbox(value=True, description='Comorbidities')
AnalysisType_widget = widgets.RadioButtons(options=['Hospital Stay', 'ICU Stay'], value='Hospital Stay', description='')

# Arranging widgets
ui = VBox([HBox([Gender_widget, AnalysisType_widget]), Comorbidities_widget])

def plot_stay(Gender, Comorbidities, AnalysisType):
    if Gender == 'Female':
        filtered_df = df_2[df_2['Gender'] == 'F']
    elif Gender == 'Male':
        filtered_df = df_2[df_2['Gender'] == 'M']
    else:
        filtered_df = df_2

    df_com = filtered_df[filtered_df['Comorbidities'] == True]
    df_com_not = filtered_df[filtered_df['Comorbidities'] == False]

    if Comorbidities:
        filtered_df = df_com
    else:
        filtered_df = df_com_not

    if AnalysisType == 'Hospital Stay':
        average_stay_by_age_group = filtered_df.groupby('Age Group')['HospitalStay'].mean().reset_index()
        y_label = 'Average Hospital Stay (Days)'
        title = 'Average Time Spent in a Hospital (days)'
        y = 'HospitalStay'
    else: # ICU Analysis
        average_stay_by_age_group = filtered_df.groupby('Age Group')['ICU Stay (Days)'].mean().reset_index()
        y_label = 'Average ICU Stay (Days)'
        title = 'Average ICU Stay'
        y = 'ICU Stay (Days)'

    plt.figure(figsize=(10,5))
    bars = sns.barplot(x='Age Group', y=y, data=average_stay_by_age_group, palette=[base_color])

    value_range = max(average_stay_by_age_group[y]) - min(average_stay_by_age_group[y]) + 0.5

    for i, bar in enumerate(plt.gca().patches):
        value = average_stay_by_age_group[y][i]
        darken_factor = (value - min(average_stay_by_age_group[y])) / value_range
        new_color = tuple(max(0, min(1, c * (1 - darken_factor))) for c in sns.color_palette([base_color])[0])
        bar.set_facecolor(new_color)
    
    for bar in bars.patches:
        bars.annotate(format(bar.get_height(), '.1f'), 
                      (bar.get_x() + bar.get_width() / 2, 
                       bar.get_height()), ha='center', va='center',
                       size=10, xytext=(0, 8),
                       textcoords='offset points',
                      bbox=dict(boxstyle="round,pad=0.2", facecolor='grey',alpha=0.2))  # Bounding box


    plt.tight_layout()

    plt.title(title)
    plt.xlabel('Age Group')
    # plt.xticks(rotation=45, fontsize=12)

    plt.ylabel(y_label)
    plt.show()


out = interactive_output(plot_stay, {'Gender': Gender_widget, 'Comorbidities': Comorbidities_widget, 'AnalysisType': AnalysisType_widget})

display(ui, out)

Output()

In [9]:
# Here I'm using <blockquote> for semantic emphasis and <strong> to highlight the age specification.
txt = widgets.HTML(
    value="""<blockquote><a href="https://www.strokebestpractices.ca/recommendations/acute-stroke-management/triage-and-initial-diagnostic-evaluation-of-transient-ischemic-attack-and-non-disabling-stroke" target=_blank style="color: blue">(Reference) Acute Stroke Management (Section 2.1 Clinical Considerations)</a> <br> According to Canadian Best Stroke Practices, referral to a healthcare professional 
    with stroke expertise should be considered for patients with a suspected uncommon cause of stroke, 
    including for <strong>young patients with stroke (e.g., &lt;45 years of age)</strong>.</blockquote>""",
    layout=widgets.Layout(margin='10px', padding='10px', border='2px solid red', width='auto')
)

# Display the widget in a layout that stands out.
display(txt)


HTML(value='<blockquote><a href="https://www.strokebestpractices.ca/recommendations/acute-stroke-management/tr…

## ICU Visits Analysis

In [10]:
df_3 = df.copy()

In [11]:
df_icu = df_3[df_3['ICU Checkout Time'].notna()]
df_icu_visit = df_icu[df_icu.isin(['Neurologist Visit']).notna().any(axis=1)].copy()
visits = ['Neurologist Visit', 'Occupational Therapist Visit', 'Speech Pathologist Visit', 'Physiotherapist Visit', 'Dietitian Visit', 'Social Worker Visit', 'Cardiologist Visit']

visit_widget = widgets.SelectMultiple(options=visits, value=['Neurologist Visit'], description='Visits(Multiple):', style={'description_width': 'initial'})
comorbidities_widget = widgets.Checkbox(value=False, description='Comorbidities')
viewBy_widget = widgets.RadioButtons(options=['Patient Id', 'Age'], value='Patient Id', description='View By:')

# Arranging widgets
ui = VBox([HBox([visit_widget, viewBy_widget]), comorbidities_widget])

def plot_visits(Visit, ViewBy, Show_Comorbidities):
    fig, ax = plt.subplots(figsize=(10,6))

    if Show_Comorbidities:
        df_icu_visit = df_icu[df_icu['Comorbidities']]

    df_icu_visit = df_icu[df_icu.isin(Visit).notna().any(axis=1)].copy()
    
    for visit in Visit:
        doctor_visit = pd.to_datetime(df_icu_visit[visit])
        icu_time = pd.to_datetime(df_icu_visit['ICU Checkout Time'])

        df_icu_visit[f'Arrival in ICU ({visit})'] = doctor_visit - icu_time


        df_icu_visit.loc[df_icu_visit[f'Arrival in ICU ({visit})'] <= pd.Timedelta(0), f'In ICU ({visit})'] = 'Visit During\nICU Stay'
        df_icu_visit.loc[df_icu_visit[f'Arrival in ICU ({visit})'] > pd.Timedelta(0), f'In ICU ({visit})'] = 'Visit After\nICU Checkout'

    df_icu_visit['Intersection'] = np.nan

    df_icu_visit['Intersection'] = df_icu_visit['Intersection'].astype(object)

    for visit in Visit:
        df_icu_visit.loc[df_icu_visit[f'In ICU ({visit})'] == 'Visit During\nICU Stay', 'Intersection'] = 'Visit During\nICU Stay'
        df_icu_visit.loc[df_icu_visit[f'In ICU ({visit})'] == 'Visit After\nICU Checkout', 'Intersection'] = 'Visit After\nICU Checkout'

    nan_condition = df_icu_visit[[f'In ICU ({visit})' for visit in Visit]].isna().any(axis=1)
    df_icu_visit.loc[nan_condition, 'Intersection'] = np.nan

    project_palette = {'Visit During\nICU Stay': RED, 'Visit After\nICU Checkout': BLUE}

    sns.stripplot(data=df_icu_visit, x=ViewBy, y='Intersection', hue='Intersection', s=2, order=['Visit During\nICU Stay', 'Visit After\nICU Checkout'], ax=ax, legend=False, palette=project_palette)

    ax.set_ylabel('')
    ax.set_title('ICU Visits Analysis')


out = interactive_output(plot_visits, {'Visit': visit_widget, 'Show_Comorbidities': comorbidities_widget, 'ViewBy': viewBy_widget})

display(ui, out)


Output()

In [12]:
txt = widgets.HTML(
    value="""<blockquote> <a href = "https://prod.strokebestpractices.ca/recommendations/acute-stroke-management/inpatient-prevention-and-management-of-complications-following-stroke?_ga=2.101666051.384248697.1711740394-1300068819.1711740393" target=_blank style="color: blue">(Reference) Acute Stroke Management (9.6 Nutrition and Dysphagia(iii))</a> <br>

Abnormal results from the initial or ongoing swallowing screens should trigger a prompt referral to a <strong>speech-language pathologist, occupational therapist, dietitian,</strong> and/or other trained dysphagia clinicians for more detailed assessment and management of swallowing, feeding, nutritional, and hydration status [Strong recommendation; Moderate quality of evidence].
</blockquote>""",
    layout=widgets.Layout(margin='10px', padding='10px', border='2px solid red', width='auto')
)

display(txt)

HTML(value='<blockquote> <a href = "https://prod.strokebestpractices.ca/recommendations/acute-stroke-managemen…

In [13]:
# When selecting the Occupational Therapist Visit, Speech Pathalogist Visit and Dietician Visit together and setting x-axis to Age, we came to know that patients from <strong>50 to 90 years</strong> are being referred to these physicians during their stay in ICU, probably showing the mentioned symptoms during their ICU stay.

# We expected the neurologist visit during ICU time the most but it seems that Neurologist visit after the ICU checkout is also significant.


## TPA Impact Analysis

In [14]:
df_4 = df.copy()

In [17]:
df_4= df_4[df_4['Comorbidities']==False]

df_4['TPA Time'] = pd.to_datetime(df_4['TPA Time'])
df_4['Emergency Dept Time'] = pd.to_datetime(df_4['Emergency Dept Time'])
df_4['TPA_Done'] = df_4['TPA Time'].notna().astype(int)
df_4['TimeForTPA'] = (df_4['TPA Time'] - df_4['Emergency Dept Time']).dt.total_seconds() / 60

df_4['ICU Checkout Time'] = pd.to_datetime(df_4['ICU Checkout Time'])
df_4['ICU Arrival Time'] = pd.to_datetime(df_4['ICU Arrival Time'])
df_4['ICUStay(Days)'] = (df_4['ICU Checkout Time'] - df_4['ICU Arrival Time']).dt.total_seconds() / (60 * 60 * 24)
df_4['ICUStay_Flag'] = df_4['ICU Arrival Time'].notna().astype(int)
df_4['Admission Time'] = pd.to_datetime(df_4['ICU Checkout Time'])
df_4['ICU Arrival Time'] = pd.to_datetime(df_4['ICU Arrival Time'])
df_4['ICUStay(Days)'] = (df_4['ICU Checkout Time'] - df_4['ICU Arrival Time']).dt.total_seconds() / (60 * 60 * 24)

def categorize_age(age):
    if age <= 45:
        return '0-45'
    elif 45 < age <= 60:
        return '45-60'
    elif 60 < age <=70:
        return '60-70'
    elif 70 < age <= 80:
        return '70-80'
    else:
        return '80+'

# Apply the function to create a new column 'Age Group'
df_4['Age Group'] = df_4['Age'].apply(categorize_age)

In [19]:
age_groups = sorted(df_4['Age Group'].unique())
color_mapping = {
    (0, 0): '#2ca02c',  # Non-ICU & No TPA # green
    (0, 1): OCHRE,  # Non-ICU & TPA - ochre
    (1, 0): BLUE,  # ICU & No TPA green
    (1, 1): RED,  # ICU & TPA red
}

@interact
def plot_treemap(selected_age_groups=widgets.SelectMultiple(options=age_groups, value=age_groups, description="Age Groups")):
    filtered_df = df_4[df_4['Age Group'].isin(selected_age_groups)]
    
    icu_tpa = filtered_df.groupby(['ICUStay_Flag', 'TPA_Done']).size().reset_index(name='counts')
    icu_tpa['label'] = icu_tpa.apply(lambda x: f"{'TPA' if x['TPA_Done'] == 1 else 'No TPA'} --> {'ICU' if x['ICUStay_Flag'] == 1 else 'No ICU'}\nPatients: {x['counts']}", axis=1)
    sizes = icu_tpa['counts'].values
    labels = icu_tpa['label'].values

    colors = [color_mapping[(row['ICUStay_Flag'], row['TPA_Done'])] for index, row in icu_tpa.iterrows()]

    plt.figure(figsize=(10, 8))
    squarify.plot(sizes=sizes, label=labels, alpha=0.8, color=colors, ec='w', text_kwargs=dict(color='#002147', fontweight='bold'))
    plt.title('TPA Impact Analysis')
    plt.axis('off')
    plt.show()


interactive(children=(SelectMultiple(description='Age Groups', index=(0, 1, 2, 3, 4), options=('0-45', '45-60'…

## Attributions

- Widget List - https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html
- Time Series / date functionality - https://pandas.pydata.org/docs/user_guide/timeseries.html
- Color Inspiration - https://crsn.ca/en/stroke-best-practices/best-practice-guidelines
- StripPlot - https://seaborn.pydata.org/generated/seaborn.stripplot.html#seaborn.stripplot
- Indexing Copy - https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
- Changing Color in Seaborn Bar plot - https://stackoverflow.com/questions/36271302/changing-color-scale-in-seaborn-bar-plot
- Squarify - https://github.com/laserson/squarify